<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/Practice_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization in Neural Networks

# keywords: stochastic gradient decent, momentum, adagrad, adam

# SGD optimizer

In [0]:
import torch

This is a sample implementation of SGD optimizer.

In [0]:
class MySGD(torch.optim.Optimizer):  
  def __init__(self, params, lr):
    self._lr = lr
    defaults = {}
    super().__init__(params, defaults)

  @torch.no_grad()
  def step(self):
    for group in self.param_groups:
      for p in group['params']:
        if p.grad is None:
          continue
        d_p = p.grad
        p.add_(d_p, alpha=-self._lr)
    return loss

Let's test that it works correctly (the code below is copied from Practice 2).

In [3]:
from torchvision import datasets, transforms
train_dataset = datasets.MNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
test_dataset = datasets.MNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
import torch.nn as nn
from torch.nn.modules import loss

In [5]:
class Flatten(nn.Module):
  def forward(self, x):
    batch_size = x.shape[0]
    return x.view(batch_size, -1)

def get_model():
  return nn.Sequential(Flatten(), 
                       nn.Linear(784, 512), 
                       nn.Tanh(),
                       nn.Linear(512, 64), 
                       nn.Tanh(),
                       nn.Linear(64, 10))

loss_function = loss.CrossEntropyLoss()

def train(model, train_loader, optimizer, loss_function, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader, loss_function):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [0]:
model = get_model()
optim = MySGD(model.parameters(), lr=0.1)

In [0]:
for epoch in range(1, 5):
  train(model, train_loader, optim, loss_function, epoch)
  test(model, test_loader, loss_function)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339847
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.268448
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.319674
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.252545
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.214682

Test set: Average loss: 0.0028, Accuracy: 9493/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.213813
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.185647
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.087889
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.195175
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.213207

Test set: Average loss: 0.0019, Accuracy: 9632/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.088355
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.079265
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.046103
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.037328
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.077569


Why do we need defaults?

In [0]:
class MySGD(torch.optim.Optimizer):  
  def __init__(self, params, lr):
    defaults = {"lr": lr}
    super().__init__(params, defaults)

  @torch.no_grad()
  def step(self):
    for group in self.param_groups:
      for p in group['params']:
        if p.grad is None:
          continue
        d_p = p.grad
        p.add_(d_p, alpha=-group["lr"])
    return loss

In [0]:
model = get_model()
model

In [0]:
optim = MySGD([{'params': model[:2].parameters(), 'lr': 0.01}, 
               {'params': model[2:].parameters()}], lr = 0.1)

In [0]:
for epoch in range(1, 5):
  train(model, train_loader, optim, loss_function, epoch)
  test(model, test_loader, loss_function)

Let's modify our SGD class to introduce an exponential momentum.

In [0]:
class ExpMomentumSGD(torch.optim.Optimizer):  
  def __init__(self, params, lr, momentum):
    defaults = {"lr": lr, "momentum": momentum}
    super().__init__(params, defaults)

  @torch.no_grad()
  def step(self):
    for group in self.param_groups:
      momentum = group['momentum']

      for p in group['params']:
        param_state = self.state[p]

        if p.grad is None:
          continue
        d_p = p.grad
        if 'momentum_buffer' not in param_state:
          buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
        else:
          buf = param_state['momentum_buffer']
          buf.mul_(momentum).add_(d_p)
        d_p = buf
        p.add_(d_p, alpha=-group["lr"])
    return loss

In [0]:
model = get_model()
optim = ExpMomentumSGD(model.parameters(), lr=0.1, momentum=0.9)

In [0]:
for epoch in range(1, 5):
  train(model, train_loader, optim, loss_function, epoch)
  test(model, test_loader, loss_function)

Linear momentum

In [0]:
class LinearMomentumSGD(torch.optim.Optimizer):  
  def __init__(self, params, lr, momentum):
    defaults = {"lr": lr, "momentum": momentum}
    super().__init__(params, defaults)

  @torch.no_grad()
  def step(self):
    for group in self.param_groups:
      momentum = group['momentum']

      for p in group['params']:
        param_state = self.state[p]

        if p.grad is None:
          continue
        d_p = p.grad
        if 'momentum_buffer' not in param_state:
          ...
        d_p = buf
        p.add_(d_p, alpha=-group["lr"])
    return loss

Nesterov momentum



In [0]:
class SGD(Optimizer):
    r"""Implements stochastic gradient descent (optionally with momentum).
    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf
    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.
        Considering the specific case of Momentum, the update can be written as
        .. math::
            \begin{aligned}
                v_{t+1} & = \mu * v_{t} + g_{t+1}, \\
                p_{t+1} & = p_{t} - \text{lr} * v_{t+1},
            \end{aligned}
        where :math:`p`, :math:`g`, :math:`v` and :math:`\mu` denote the 
        parameters, gradient, velocity, and momentum respectively.
        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form
        .. math::
            \begin{aligned}
                v_{t+1} & = \mu * v_{t} + \text{lr} * g_{t+1}, \\
                p_{t+1} & = p_{t} - v_{t+1}.
            \end{aligned}
        The Nesterov version is analogously modified.
    """

    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(d_p, alpha=1 - dampening)
                    if nesterov:
                        d_p = d_p.add(buf, alpha=momentum)
                    else:
                        d_p = buf

                p.add_(d_p, alpha=-group['lr'])

        return loss

# Tensorboard

The easiest way to compare different models/optimizers/initialization is to plot loss after each epoch.

In [0]:
from torch.utils.tensorboard import SummaryWriter

In [0]:
! rm -r logs logs2 logs3

In [0]:
writer = SummaryWriter(log_dir="logs")

In [0]:
writer.add_scalar("plot1", torch.tensor(1.0), global_step = 1)

In [0]:
writer.add_scalar("plot1", torch.tensor(3.0), global_step = 2)

In [0]:
writer.add_scalar("plot1", torch.tensor(2.0), global_step = 3)

In [0]:
writer.close()

In [0]:
%load_ext tensorboard


In [0]:
%tensorboard --logdir logs


In [0]:
writer = SummaryWriter(log_dir="logs")
writer.add_scalar("plot2", torch.tensor(1.0), global_step = 1)
writer.add_scalar("plot2", torch.tensor(4.0), global_step = 2)
writer.add_scalar("plot2", torch.tensor(5.0), global_step = 3)
writer.close()

How to draw multiple line on the same plot?

In [0]:
writer = SummaryWriter(log_dir="logs2/experiment1")
writer.add_scalar("plot2", torch.tensor(1.0), global_step = 1)
writer.add_scalar("plot2", torch.tensor(3.0), global_step = 2)
writer.add_scalar("plot2", torch.tensor(2.0), global_step = 3)
writer.close()
writer = SummaryWriter(log_dir="logs2/experiment2")
writer.add_scalar("plot2", torch.tensor(1.0), global_step = 1)
writer.add_scalar("plot2", torch.tensor(4.0), global_step = 2)
writer.add_scalar("plot2", torch.tensor(5.0), global_step = 3)
writer.close()

In [0]:
%tensorboard --logdir logs2

What else can we plot?

In [0]:
writer = SummaryWriter(log_dir="logs3")
writer.add_image("mnist_sample", train_dataset[0][0], global_step=1)
writer.add_image("mnist_sample", train_dataset[1][0], global_step=2)
writer.close()

In [0]:
%tensorboard --logdir logs3

In [0]:
writer = SummaryWriter(log_dir="logs4")
writer.add_graph(model, train_loader.__iter__().__next__()[0])


In [0]:
writer.close()

In [0]:
%tensorboard --logdir logs4

In [0]:
%tensorboard --logdir logs5

In [0]:
writer = SummaryWriter(log_dir="logs5")

In [0]:
import time

In [0]:
for epoch in range(10):
  time.sleep(15)
  writer.add_scalar("linear", torch.tensor(epoch * 2), global_step=epoch)
  writer.add_scalar("parabolic", torch.tensor(epoch ** 2), global_step=epoch)
  if epoch % 2 == 0:
    writer.add_image("mnist", train_dataset[epoch][0], global_step=epoch)
writer.close()

In [0]:
%tensorboard --logdir logs6

In [0]:
for epoch in range(100):
  writer = SummaryWriter(log_dir="logs6")
  time.sleep(5)
  writer.add_scalar("linear", torch.tensor(epoch * 2), global_step=epoch)
  writer.add_scalar("parabolic", torch.tensor(epoch ** 2), global_step=epoch)
  if epoch % 2 == 0:
    writer.add_image("mnist", train_dataset[epoch][0], global_step=epoch)
  writer.close()

# Assignment [10]

1. Update training and test function in such a way that train loss is ploted on "train_loss" graph, test loss on "test_loss" graph and accuracy on the test set on "test_accuracy" graph. [2]
2. Every 5 epochs choose 9 images from test_set which are missclassified and display them in tensorboard (use add_images function of writer https://pytorch.org/docs/stable/tensorboard.html).[2]
3. Choose 5 different gradient-decent optimizers (do not choose SGD) https://pytorch.org/docs/stable/optim.html, train model with given optimizer and with default parameters. Plot train losses on "train_loss" graph, test losses on "test_loss" graph, accuracy on "test_accuracy" graph. Train at least for 50 epochs. [2]
4. In PyTorch implementation of nesterov momentum differs from one you saw at lecture. Show that it still works in the same way up to one step. [2]
5. Train your model with LBFGS optimizer https://pytorch.org/docs/stable/optim.html. Choose history size as large as you can fit in the memory. **Impotant** you'll need to call `step` with clojure parameter. Find how to use it at https://pytorch.org/docs/stable/optim.html[2].